In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# %% load packages
import locale
import sys
import os
import pandas as pd
import numpy as np
import polars as pl
import matplotlib.pyplot as plt
import optuna
import requests
import torch
from torch import nn, optim
from torch.utils.data import TensorDataset, DataLoader
import random
from sqlalchemy import create_engine,inspect
from pathlib import Path
import urllib.parse
import pyarrow
from calendar import day_abbr
import calendar
from typing import Tuple, Union, Dict, List
from concurrent.futures import ThreadPoolExecutor, as_completed
from pygam import LinearGAM
from datetime import datetime




/opt/anaconda3/envs/dl-energy-env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
from srs.utils.tutor_utils import prepare_dataset_tensor, forecasting_study,\
  plot_daily_profile,plot_hour_comparison, build_multiwindow_experts, tune_ewa_eta, \
  ewa_aggregate_forecasts, compute_error_table, tune_expert_window, \
  run_expert_window_test, build_regression_matrix, prepare_train_test_tensors, \
  DST_trafo, prepare_dataset_tensor_modified

from srs.utils.our_utils import run_forecast_step
from srs.collect_data.setup import setup_seed, get_device
from srs.collect_data.entsoe_data import create_entsoe_engine, get_tables, get_spec, \
  get_market_divisions,get_map_codes,get_map_codes_starting_with, get_resolution_codes, \
    prepare_generation, prepare_load,prepare_price, prepare_unavailability, \
    prepare_filling_rate_hydro, prepare_physical_flow, prepare_installed_capacity
from srs.collect_data.datastream_data import create_datastream_engine, get_tables, \
  prepare_datastream
from srs.collect_data.dwd_mosmix_data import fetch_region_weather, prepare_weather
from srs.collect_data.merge_data import merge_datasets, build_training_dataset
from srs.models.mlp import SimpleMLP, train_mlp, build_mlp_rolling_forecasts, \
  tune_mlp_hyperparameters, DeepMLP


In [ ]:
'''
  training interval:
  2019 - 365 days
  2020 - 366 days
  2021 - 365 days
  2022 - 366 days
  
  testing interval:
  2023 - 365 days
  2024 - 366 days
  
  
The reason why I have metnioned lightgbm and GAM before is that I have, as one of the alternative methodologies, to get preliminary predictions from gam,lightbgm or any other models, use these predictions as a input for a input layer to get final predictions from MLP.
That is why I need to be consistent for now with 
'''

In [ ]:
# 13. Tutorial 5. MLP  — rolling-window expert + Optuna tuning

# -------------------------------------------------------------
# 0) Regression matrix on *all* data (no NaNs)
reg_data = build_regression_matrix(
    dat_eval = train_t.cpu().numpy(),
    days_eval= pd.to_datetime(train_dates),
    reg_names= df.columns[1:],   # all columns except time_utc
)
reg_df   = reg_data["regmat"].dropna().reset_index(drop=True)
dep_idx  = reg_data["dep_indices"]

# -------------------------------------------------------------
# 1)  Optuna tuning on a 730-day evaluation block
eval_start  = reg_df.shape[0] - 730          # first eval row
best_params, study = tune_mlp_hyperparameters(
    reg_df, dep_idx, eval_window = (eval_start, 730), n_trials = 40
)
print("best params:", best_params)

# -------------------------------------------------------------
# 2)  Build rolling forecasts on a 730-day *test* block
test_horizon = 730
test_start   = reg_df.shape[0] - test_horizon
preds_mlp, trues_mlp = build_mlp_rolling_forecasts(
    reg_df, dep_idx,
    window      = best_params["D"],
    horizon     = test_horizon,
    start_row   = test_start,
    hidden_dim  = best_params["hidden"],
    lr          = best_params["lr"],
    weight_decay= best_params["wd"],
)

### MLP with rolling window and without Optuna

In [ ]:
# global constants to all zones
INIT_DATE_EXPERIMENTS = '2019-01-01'
INIT_TEST_DATE        = '2023-01-01'
FINAL_DATE_EXPERIMENTS= '2024-12-31'

# hyperparameters
WINDOW_DAYS   = 730                 
HIDDEN_DIM    = 50                  
LEARNING_RATE = 1e-3
WEIGHT_DECAY  = 1e-3
EPOCHS        = 60
BATCH_SIZE    = 32

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

repo_root  = Path.cwd().parents[1]
mapcodes   = ["NO1", "NO2", "NO3", "NO4", "NO5"]
zone_data  = {}          

for code in mapcodes:
    csv_path = repo_root / "data" / f"data_{code}.csv"
    df_raw   = pd.read_csv(csv_path, parse_dates=["time_utc"])

    data_t, train_t, train_dates, price_t = prepare_dataset_tensor_modified(
        csv_path,
        tz      = "CET",
        seed    = 42,
        test_days = (pd.Timestamp(FINAL_DATE_EXPERIMENTS)
                     - pd.Timestamp(INIT_TEST_DATE)).days + 1,
        dtype   = torch.float64,
    )

    idx = pd.DatetimeIndex(sorted(train_dates))
    start_i = idx.get_loc(pd.Timestamp(INIT_DATE_EXPERIMENTS))
    end_i   = idx.get_loc(pd.Timestamp(FINAL_DATE_EXPERIMENTS))
    data_t  = data_t[start_i:end_i+1]             
    dates_t = pd.Series(train_dates[start_i:end_i+1])

    zone_data[code] = dict(
        df        = df_raw,
        tensor    = data_t,
        dates     = dates_t,
        price_t   = price_t[start_i:end_i+1],
    )

# rolling-window MLP per zone
rmse_mlp_by_zone   = {}
preds_mlp_by_zone  = {}

for code in mapcodes:
    print(f"\n==== Zone {code} ====")

    reg_data = build_regression_matrix(
        dat_eval = zone_data[code]["tensor"].cpu().numpy(),
        days_eval= pd.to_datetime(zone_data[code]["dates"]),
        reg_names= zone_data[code]["df"].columns[1:],   
    )
    reg_df   = reg_data["regmat"].dropna().reset_index(drop=True)
    dep_idx  = reg_data["dep_indices"]

    all_dates = pd.DatetimeIndex(sorted(zone_data[code]["dates"]\
                                        .iloc[len(zone_data[code]["dates"])
                                             - len(reg_df):]))
    test_start_row = all_dates.get_loc(pd.Timestamp(INIT_TEST_DATE))
    test_end_row   = all_dates.get_loc(pd.Timestamp(FINAL_DATE_EXPERIMENTS))
    horizon        = test_end_row - test_start_row + 1      

    preds, trues = build_mlp_rolling_forecasts(
        regmat_df   = reg_df.astype("float32"),
        dep_indices = dep_idx,
        window      = WINDOW_DAYS,
        horizon     = horizon,
        start_row   = test_start_row,
        hidden_dim  = HIDDEN_DIM,
        lr          = LEARNING_RATE,
        weight_decay= WEIGHT_DECAY,
        batch_size  = BATCH_SIZE,
        epochs      = EPOCHS,
        device      = device,
    )

    rmse = torch.sqrt(((preds - trues) ** 2).mean()).item()
    rmse_mlp_by_zone[code]  = rmse
    preds_mlp_by_zone[code] = preds                   

    print(f"RMSE 2023-24: {rmse:7.3f}")

print("\n===== Rolling-MLP RMSE ( NOK / MWh ) =====")
for z, r in rmse_mlp_by_zone.items():
    print(f"{z}:  {r:7.3f}")


In [ ]:
# -------------------------------------------------------------
# 3)  Append to forecast_all and compute error table
mlp_chan  = preds_mlp.unsqueeze(2)            # (N,24,1)
forecast_all = torch.cat([forecast_all, mlp_chan], dim=2)
model_names  = model_names + ["MLP"]

err_table_with_mlp = compute_error_table(forecast_all, model_names)
print(err_table_with_mlp)


Zone NO1
RMSE 2023-24:  21.057

Zone NO2
RMSE 2023-24:  24.377

Zone NO3
RMSE 2023-24:  15.700

Zone NO4
RMSE 2023-24:  11.912

Zone NO5
RMSE 2023-24:  17.403

# Deep MLP – Rolling‑window Benchmark

## 1  Dataset & Split

* **Zones:** NO1 – NO5 (one CSV per zone, columns identical).
* **Rows:** one‑hour resolution aggregated to **one row = one day** by `build_regression_matrix`.
* **Train interval:** **2019‑01‑01 – 2022‑12‑31**.
* **Test interval:** **2023‑01‑01 – 2024‑12‑31** (730 days).
* **Rolling window:** **D = 730 days** – for every test day the model re‑trains on the 730 days that *immediately precede* it.

## 2  Feature Engineering

### Per‑hour block (repeated 24×)

```
Price_s{h}               current‑day spot price (target)   
Price_lag_{1,2,7}_s{h}   spot price lags (1,2,7 days)      
{Load,Solar,WindOn,WindOff}_DA_lag_0_s{h}  day‑ahead quantities (no lag)  
```

*Total per‑hour columns: 8  →  24 × 8 = 192.*

### Global block (shared by all hours)

```
WD_{1‑7}                       weekday dummies (Mon…Sun)   
{Coal,NGas,Oil,EUA}_lag_2      fuel & EUA prices, 2‑day lag
```

*Columns: 7 + 4 = 11.*

**Total feature width:** 192 + 11 = 203.

## 3  Model Architecture – `DeepMLP`

```
Input  (203) ➜ Linear(203, **128**) ➜ LeakyReLU ➜ Dropout(p=0.10)
            ➜ Linear(128, 128)      ➜ LeakyReLU ➜ Dropout(p=0.10)
            ➜ Linear(128, **24**)   (one output per hour)
```

* **Depth:** 2 hidden layers (`n_hidden = 2`).
* **Activation:** LeakyReLU (slope = 0.01, PyTorch default).
* **Dropout:** 10 % after each activation.
* **Parameter count:** `(203×128+128) + (128×128+128) + (128×24+24)` ≈ **54 k**.

## 4  Training Loop (executed *per test day*)

| step                  | detail                                                                            |
| --------------------- | --------------------------------------------------------------------------------- |
| **Standardise X & y** | z‑score using the current 730‑day window (mean/std computed *within* the window). |
| **Optimizer**         | `torch.optim.Adam(lr = 1e‑3, weight_decay = 1e‑3)`                                |
| **Batch size**        | 32 (full‑batch ≈ 730 rows also viable)                                            |
| **Epochs**            | 60                                                                                |
| **Loss**              | Mean‑squared error on the 24‑dim vector                                           |
| **Prediction**        | after training, predict the *next* day, then un‑standardise                       |

## 5  Evaluation Metric

* **RMSE (NOK/MWh)** averaged over the 730 test days × 24 hours.
* 2023‑24 scores observed so far:

  * NO1 21.06
  * NO2 24.38
  * NO3 15.70
  * NO4 11.91
  * NO5 17.40
    → **Average 18.10** (baseline LightGBM ≈ 8–9).

## 6  Next planned extensions

1. **Hour‑specific output heads** (shared trunk, 24 linear tails).
2. **Learning‑rate schedule** (cosine decay).
3. Hyper‑parameter sweep (`hidden_dim`, dropout, weight‑decay) with Optuna.

In [ ]:
# global constants to all zones
INIT_DATE_EXPERIMENTS = '2019-01-01'
INIT_TEST_DATE        = '2023-01-01'
FINAL_DATE_EXPERIMENTS= '2024-12-31'

# hyperparameters
WINDOW_DAYS   = 730                 
HIDDEN_DIM    = 128 # initial - 50                  
LEARNING_RATE = 1e-3
WEIGHT_DECAY  = 1e-3
EPOCHS        = 60
BATCH_SIZE    = 256

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

repo_root  = Path.cwd().parents[1]
mapcodes   = ["NO1", "NO2", "NO3", "NO4", "NO5"]
zone_data  = {}          

for code in mapcodes:
    csv_path = repo_root / "data" / f"data_{code}.csv"
    df_raw   = pd.read_csv(csv_path, parse_dates=["time_utc"])

    data_t, train_t, train_dates, price_t = prepare_dataset_tensor_modified(
        csv_path,
        tz      = "CET",
        seed    = 42,
        test_days = (pd.Timestamp(FINAL_DATE_EXPERIMENTS)
                     - pd.Timestamp(INIT_TEST_DATE)).days + 1,
        dtype   = torch.float64,
    )

    idx = pd.DatetimeIndex(sorted(train_dates))
    start_i = idx.get_loc(pd.Timestamp(INIT_DATE_EXPERIMENTS))
    end_i   = idx.get_loc(pd.Timestamp(FINAL_DATE_EXPERIMENTS))
    data_t  = data_t[start_i:end_i+1]             
    dates_t = pd.Series(train_dates[start_i:end_i+1])

    zone_data[code] = dict(
        df        = df_raw,
        tensor    = data_t,
        dates     = dates_t,
        price_t   = price_t[start_i:end_i+1],
    )

# rolling-window MLP per zone
rmse_dmlp_by_zone   = {}
preds_dmlp_by_zone  = {}

for code in mapcodes:
    print(f"\n==== Zone {code} ====")

    reg_data = build_regression_matrix(
        dat_eval = zone_data[code]["tensor"].cpu().numpy(),
        days_eval= pd.to_datetime(zone_data[code]["dates"]),
        reg_names= zone_data[code]["df"].columns[1:],   
    )
    reg_df   = reg_data["regmat"].dropna().reset_index(drop=True)
    dep_idx  = reg_data["dep_indices"]

    all_dates = pd.DatetimeIndex(sorted(zone_data[code]["dates"]\
                                        .iloc[len(zone_data[code]["dates"])
                                             - len(reg_df):]))
    test_start_row = all_dates.get_loc(pd.Timestamp(INIT_TEST_DATE))
    test_end_row   = all_dates.get_loc(pd.Timestamp(FINAL_DATE_EXPERIMENTS))
    horizon        = test_end_row - test_start_row + 1      

    preds, trues = build_mlp_rolling_forecasts(
        regmat_df   = reg_df.astype("float32"),
        dep_indices = dep_idx,
        window      = WINDOW_DAYS,
        horizon     = horizon,
        start_row   = test_start_row,
        hidden_dim  = HIDDEN_DIM,
        lr          = LEARNING_RATE,
        weight_decay= WEIGHT_DECAY,
        batch_size  = BATCH_SIZE,
        epochs      = EPOCHS,
        device      = device,
    )

    rmse_dmlp = torch.sqrt(((preds - trues) ** 2).mean()).item()
    rmse_dmlp_by_zone[code]  = rmse_dmlp
    preds_dmlp_by_zone[code] = preds                   

    print(f"RMSE 2023-24: {rmse_dmlp:7.3f}")

print("\n Rolling-MLP RMSE")
for z, r in rmse_dmlp_by_zone.items():
    print(f"{z}:  {r:7.3f}")



==== Zone NO1 ====


/opt/anaconda3/envs/dl-energy-env/lib/python3.11/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


In [ ]:
#------------------------------
# Try this only in Collab:
#------------------------------

# build one big training DataLoader spanning entire training window
full_loader = DataLoader(
    TensorDataset(
        regmat[: train_end],
        dep_var[: train_end]
    ),
    batch_size=batch_size,
    shuffle=True,
    pin_memory=True,
    num_workers=4,
)

# instantiate & train ONE model
model = DeepMLP(...).to(device)
opt   = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
loss_fn = nn.MSELoss()

for epoch in range(epochs):
    model.train()
    for xb, yb in full_loader:
        opt.zero_grad()
        loss_fn(model(xb), yb).backward()
        opt.step()

# roll forward window and predict all horizon in one go
model.eval()
with torch.no_grad():
    preds = []
    for n in range(horizon):
        idx = test_start + n
        mean_x = regmat[idx-window:idx].mean(0, keepdim=True)
        std_x  = regmat[idx-window:idx].std(0, keepdim=True).clamp_min(1)
        x_test = ((regmat[idx:idx+1] - mean_x) / std_x)
        pred_std = model(x_test)
        mean_y = dep_var[idx-window:idx].mean(0, keepdim=True)
        std_y  = dep_var[idx-window:idx].std(0, keepdim=True).clamp_min(1)
        preds.append(pred_std * std_y + mean_y)
    preds = torch.cat(preds, dim=0)
